In [24]:
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [25]:
# Remove metadata and header rows
file_path = 'data/cumulative_2024.03.14_10.24.31.csv'

data_rows_clean = []

with open(file_path, mode='r', newline='') as file:
    reader = csv.reader(file)
    # Skip the first 53 lines (headers and metadata)
    for _ in range(53):
        next(reader)
    # Now read the actual data
    data_rows_clean = [row for row in reader]

# Convert to dataframe
df_cumulative = pd.DataFrame(data_rows_clean[1:], columns=data_rows_clean[0])

# Convert koi_disposition false positive to 0, candidate to 1, confirmed to 2
df_cumulative['koi_disposition'] = df_cumulative['koi_disposition'].map({'FALSE POSITIVE': 0, 'CANDIDATE': 1, 'CONFIRMED': 2})

# Convert koi_pdisposition false positive to 0, candidate to 1
df_cumulative['koi_pdisposition'] = df_cumulative['koi_pdisposition'].map({'FALSE POSITIVE': 0, 'CANDIDATE': 1})

# Convert columns to float
non_numeric_columns = ['kepid', 'kepoi_name', 'kepler_name', 'koi_disposition', 'koi_pdisposition', 'koi_tce_delivname']
numeric_columns = [col for col in df_cumulative.columns if col not in non_numeric_columns]

for column in numeric_columns:
    df_cumulative[column] = pd.to_numeric(df_cumulative[column], errors='coerce')

# Remove error columns
columns_to_drop = [column for column in df_cumulative.columns if 'err' in column]
df_cumulative = df_cumulative.drop(columns=columns_to_drop, axis=1)

# Remove rows that have empty koi_impact or koi_duration values
df_cumulative = df_cumulative.dropna(subset=['koi_impact', 'koi_duration'])


# Remove rows that have flags set to 1
flags = ['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec']
df_cumulative = df_cumulative[(df_cumulative[flags] == 0).all(axis=1)]

# Remove flag columns
df_cumulative = df_cumulative.drop(columns=flags)

display(df_cumulative.describe())

,koi_disposition,koi_pdisposition,koi_score,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
count,4647.000000,4647.000000,4082.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4554.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000,4647.000000
mean,1.575640,0.989886,0.907249,86.257842,166.795129,0.547936,4.738035,1166.007015,44.495091,823.628362,725.939742,54.160017,1.377251,5561.298687,4.370596,1.293090,291.352629,44.194748,14.348464
std,0.514776,0.100070,0.227633,1910.111811,63.866021,2.871402,4.011881,6171.726627,1690.711783,473.580513,8285.135379,169.638422,0.778286,687.020039,0.322688,3.532783,4.792697,3.630707,1.262344
min,0.000000,0.000000,0.000000,0.241843,121.119423,0.000000,0.052000,0.000000,0.220000,25.000000,0.000000,0.000000,1.000000,2703.000000,0.114000,0.109000,280.206600,36.577381,6.974000
25%,1.000000,1.000000,0.961000,5.468943,134.090930,0.112000,2.361000,162.450000,1.360000,503.000000,15.140000,11.700000,1.000000,5223.000000,4.275000,0.816000,287.652620,41.255600,13.615000
50%,2.000000,1.000000,0.998000,13.153841,140.655700,0.386000,3.565000,353.900000,2.000000,750.000000,74.800000,19.600000,1.000000,5662.000000,4.450000,0.973000,291.529510,44.143581,14.621000
75%,2.000000,1.000000,1.000000,42.059039,171.936930,0.747500,5.684500,759.400000,2.940000,1034.000000,270.295000,38.900000,1.000000,6007.000000,4.551000,1.236000,295.228790,47.014204,15.326500
max,2.000000,1.000000,1.000000,129995.778400,907.044710,98.602100,54.520000,348130.000000,109061.000000,6285.000000,369989.680000,4304.300000,7.000000,10894.000000,5.364000,152.969000,301.720760,52.220341,17.475000


In [26]:
# convert cleaned data to csv
df_cumulative.to_csv('data/cumulative_clean.csv', index=False)

In [27]:
# Scrape data columns from the Kepler Objects of Interest html.

# Read in the html file and soup it 
with open('data/Data columns in Kepler Objects of Interest Table.html','r') as f:
    html = f.read()
    soup = BeautifulSoup(html, 'html.parser')

# "Database Column Names" Colummn
column_name = []
for element in soup.find_all('td', class_='name'):
    column_name.append(element.text.strip())

# "Table Label" Column
tabel_labels = []
for element in soup.find_all('td', class_='label'):
    text = element.text.strip()
    if text != '' and '_str' not in text:
        tabel_labels.append(element.text.strip())

# "Description" Column
description = []
for element in soup.find_all('td', class_='description'):
    description.append(element.text.strip())


# Create a dataframe
data_dict = {'Database Column Name': column_name, 
             'Table Label': tabel_labels, 
             'Description': description}

df_columns = pd.DataFrame(data_dict)

# Remove the † from the column names
df_columns.replace('[â€]','', regex=True,inplace=True)

# Identify relavent columns that are in the cleaned data
relavent_columns = df_columns[df_columns['Database Column Name'].isin(df_cumulative.columns)]
display(relavent_columns)

,Database Column Name,Table Label,Description
0,kepid,Kepler Identification or KepID,"Target identification number, as listed in the..."
1,kepoi_name,KOI Name,A number used to identify and track a Kepler O...
2,kepler_name,Kepler Name,"Kepler number name in the form ""Kepler-N,"" plu..."
3,koi_disposition,Exoplanet Archive Disposition,The category of this KOI from the Exoplanet Ar...
6,koi_pdisposition,Disposition Using Kepler Data,The pipeline flag that designates the most pro...
7,koi_score,Disposition Score,A value between 0 and 1 that indicates the con...
14,koi_period,Orbital Period (days),The interval between consecutive planetary tra...
15,koi_time0bk,"Transit Epoch (BJD - 2,454,833.0)",The time corresponding to the center of the fi...
19,koi_impact,Impact Parameter,The sky-projected distance between the center ...
20,koi_duration,Transit Duration (hours),The duration of the observed transits. Duratio...


In [29]:
# Convert to csv file
relavent_columns.to_csv('data/Data columns in Kepler Objects of Interest Table.csv', index=False)

This research has made use of the NASA Exoplanet Archive, which is operated by the California Institute of Technology, under contract with the National Aeronautics and Space Administration under the Exoplanet Exploration Program.

DOI 10.26133/NEA4